# Model Training Notebook

In [1]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
from IPython.core.display import display, HTML
import cv2
from PIL import Image
from pathlib import Path
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

# repeatable functions housed in the utils file and imported here
from utils import *

## Create Training and Validation Datasets

https://stackoverflow.com/questions/42755214/how-to-keep-numpy-array-when-saving-pandas-dataframe-to-csv is where I got the converter code from

In [4]:
import ast
import numpy as np
def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

In [5]:
df= pd.read_csv('./datasets/annotations_map.csv', converters={'new_bb': from_np_array})

In [7]:
df_train = df.reset_index()

X = df_train[['new_path','new_bb']]
Y = df_train['class']

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
train_ds = WaldoDataset(X_train['new_path'],X_train['new_bb'] ,y_train)
valid_ds = WaldoDataset(X_val['new_path'],X_val['new_bb'],y_val)

In [12]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [13]:
cv2.imread('./images_resized/2.jpg').shape

(500, 500, 3)

## Building Model 

In [17]:
class CNN(nn.Module):
    
    ## Initialization of the model
    def __init__(self):
        super(CNN, self).__init__()
        
        ## Conv2d(in_channels, out_channels, kernel_size, stride)
        self.conv1 = nn.Conv2d(3, 32, 100, stride=10) 
        self.pool = nn.MaxPool2d(5, 2) 
        self.relu = nn.ReLU()
        self.linear= nn.Linear(11552, 4)
    
    ## Defining the forward function
    def forward(self, x):
        
        batch_size= x.shape[0]
        channel= x.shape[3]
        h= x.shape[1]
        w= x.shape[2]
        
        x= x.reshape(batch_size, channel, h, w)
        
        output = self.conv1(x)
        output = self.pool(output)
        output = self.relu(output)
        output = output.reshape(batch_size, 11552)
        output = self.linear(output)
        
        return output

# Train the Model


- Structured similarly to main.py file from pytorch part of A2

In [18]:
## Define Hyperparameters -- Currently setting values that we can modify

loss_type = "l1"
learning_rate = 1e-5
momentum = 0.9
reg = 1e-3

In [19]:
## Criterion - Loss Function
if loss_type == "CE":
    criterion = nn.CrossEntropyLoss()

if loss_type == "l1":
    criterion = nn.L1Loss()
    
if loss_type == "l2":
    criterion = nn.MSELoss()

In [20]:
model = CNN()

In [21]:
optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                            momentum=momentum,
                            weight_decay=reg)

In [30]:
def train_epocs(model, optimizer, train_dl, epochs=10):
    idx = 0
    model= model.float()
    for i in range(epochs):
        model.train()
        sum_loss = 0
        for x, y_bb in train_dl:
            x= x.float()
            out_bb = model(x)
            
            y_bb = torch.tensor(y_bb)
            
            ## out_bb and y_bb are in different formats - the need to both be tensors I think.
            ## this means we'll need to update the resize_image_bb to save the new_bb as a tensor instead of a list, I think
            loss= F.l1_loss(out_bb, y_bb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            idx += 1
            sum_loss += loss.item()
        train_loss = sum_loss
        print("Training Loss for epoch {0}: {1}".format(i,train_loss))

In [31]:
train_epocs(model= model, optimizer= optimizer, train_dl= train_dl, epochs= 5)

<ipython-input-30-687a9ae9e251>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_bb = torch.tensor(y_bb)


Training Loss for epoch 0: 1510.3412475585938
Training Loss for epoch 1: 1497.4430847167969
Training Loss for epoch 2: 1485.1582641601562
Training Loss for epoch 3: 1455.7224731445312
Training Loss for epoch 4: 1443.0083770751953
